## Getting started with geospatial data in Python
### Prepared for WiDS Brownbag; August 5, 2020 
#### by Grace Kim

In [2]:
# Import modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from scipy import spatial
from netCDF4 import Dataset 
import datetime as dt
import geopandas 

### This notebook compares data from the following sources:
#### 1. [Chesapeake Monitoring Cooperative](https://cmc.vims.edu/#/home)  
The Chesapeake Monitoring Cooperative (CMC) is a group of leading organizations that provide technical, programmatic, and outreach support in order to integrate volunteer-based and non-traditional water quality and benthic macroinvertebrate monitoring data monitoring data into the Chesapeake Bay Program partnership. 
#### 2. [North American Regional Reanalysis](https://psl.noaa.gov/data/gridded/data.narr.html)  
A long-term, consistent, high-resolution climate dataset for the North American domain. Data used in our analysis are 3-hourly average air temperature, accumulated precipitation and wind speed.

In [4]:
### Data load for Databricks; replace with your own import command if using locally 
# File location and type
file_location = "/FileStore/alldata_master.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

Date,Time,StationName,StationCode,Latitude,Longitude,GroupCode,SampleId,SampleDepth,airtemp,alkalinity,nh4,chla,conductivity,do_sat,do,entero,no3,no2,po4,ph,salinity,depth,tds,tn,tp,tss,watertemp,ntu,RainfallWithin24Hours,RainfallWithin48Hours,Rainfall,StreamFlow,TidalStage,WaterColorDescription,WaterColor,WaterSurfaces,WeatherConditionsToday,WeatherConditionsYesterday,WindSpeed,airtemp_narr,precip3_narr,precip24_narr,precip48_narr,windspeed_narr,HUC12
2015-04-21 00:00:00,09:00:00,8-PLT-36-LACA,LACA.8-PLT-36-LACA,38.143611,-77.855555,LACA,1.0,0.3,17.0,nan,nan,8.52,nan,nan,9.33,nan,nan,nan,nan,7.29,nan,1.5,nan,0.65,0.06,nan,nan,19.12,None,None,None,None,None,None,None,None,None,None,None,14.97052,0.0,4.003388,4.235404,5.208091,20801060503
2014-10-07 00:00:00,10:00:00,EX-1,LACA.EX-1,38.11488,-78.01012,LACA,1.0,0.3,nan,nan,nan,18.6,0.048,nan,6.77,nan,nan,nan,nan,6.35,nan,nan,nan,0.6,0.03,nan,nan,nan,None,None,None,None,None,None,None,None,None,None,None,14.34552,0.0,2.7768254,3.344779,5.82062,20801060402
2019-03-24 00:00:00,15:20:00,BRCR1,NWA.BRCR1,38.5645,-75.6723,NWA,1.0,0.5,nan,nan,nan,13.9,nan,119.2,11.5,nan,nan,nan,nan,nan,0.1,1.3,nan,6.21,0.05,nan,nan,11.1,None,None,None,None,Low,None,None,Ripple,Sunny,None,None,19.64972,0.0,1.519013,2.7265177,6.236149,20801090205
2017-04-09 00:00:00,17:40:00,COBR1,NWA.COBR1,38.6422,-75.6068,NWA,1.0,0.5,nan,nan,nan,16.21,nan,105.0,10.5,nan,nan,nan,nan,nan,0.1,1.1,nan,4.59,0.05,nan,nan,15.9,None,None,None,None,High,None,None,Ripple,Sunny,None,None,13.587219,0.0,15.3237,12.210892,13.09209,20801090404
2019-06-30 00:00:00,17:35:00,DEHE3,NWA.DEHE3,38.6647,-75.5639,NWA,1.0,0.5,nan,nan,nan,2.08,nan,92.2,7.3,nan,nan,nan,nan,nan,0.1,0.7,nan,4.228,0.035,nan,nan,27.2,None,None,None,None,High,None,None,Ripple,Sunny,None,None,10.71579,0.04910195,0.0050688675,1.5758352,2.195765,20801090404
2019-03-25 00:00:00,8:00:00,WECR1,NWA.WECR1,38.3267,-75.8665,NWA,1.0,0.5,nan,nan,nan,11.24,nan,95.8,10.7,10.0,nan,nan,nan,nan,1.1,4.5,nan,3.26,0.06,nan,nan,10.2,None,None,None,None,Outgoing (Ebb),None,None,Calm,Partly cloudy,None,None,11.065277,0.625,2.5703125,3.5765991,44.59802,20801090505
2018-05-03 00:00:00,10:00:00,MOS,SCC.MOS,38.9753,-76.4837,SCC,1.0,4.5,nan,nan,nan,24.3,nan,nan,11.15,4.0,nan,nan,nan,nan,7.9,nan,nan,0.94,0.0382,32.0,nan,14.5,None,0,None,None,None,None,None,None,SUNNY,None,None,12.686005,0.0,0.015625,0.06946033,6.663988,20600040203
2018-06-17 00:00:00,12:00:00,MAHO3,NWA.MAHO3,38.7627,-75.723,NWA,1.0,0.5,nan,nan,nan,nan,136.3,82.5,7.2,nan,nan,nan,nan,nan,nan,2.2,nan,4.15,0.1,nan,nan,21.8,None,None,None,None,None,None,None,Ripple,Sunny,None,None,7.9360046,0.1171875,5.7265625,6.3819604,60.17521,20801090303
2018-06-17 00:00:00,12:55:00,MAHO2,NWA.MAHO2,38.6882,-75.7701,NWA,1.0,0.5,nan,nan,nan,nan,nan,82.9,7.2,nan,nan,nan,nan,nan,0.1,2.1,nan,4.74,0.1,nan,nan,22.6,None,None,None,None,Outgoing (Ebb),None,None,Calm,Sunny,None,None,11.388733,0.1015625,5.8828125,7.1609836,59.897427,20801090305
2017-06-18 00:00:00,12:10:00,MAHO3,NWA.MAHO3,38.7627,-75.723,NWA,1.0,0.5,nan,nan,nan,nan,nan,69.5,5.7,nan,nan,nan,nan,nan,nan,1.5,nan,3.54,0.05,nan,nan,24.7,None,None,None,None,None,None,Normal,Calm,Sunny,None,None,2.4527283,0.0,0.035556138,0.035326965,0.3650178,20801090303


In [5]:
alldatadf = df.toPandas()
alldatadf.head()

,Date,Time,StationName,StationCode,Latitude,Longitude,GroupCode,SampleId,SampleDepth,airtemp,alkalinity,nh4,chla,conductivity,do_sat,do,entero,no3,no2,po4,ph,salinity,depth,tds,tn,tp,tss,watertemp,ntu,RainfallWithin24Hours,RainfallWithin48Hours,Rainfall,StreamFlow,TidalStage,WaterColorDescription,WaterColor,WaterSurfaces,WeatherConditionsToday,WeatherConditionsYesterday,WindSpeed,airtemp_narr,precip3_narr,precip24_narr,precip48_narr,windspeed_narr,HUC12
0,2015-04-21 00:00:00,09:00:00,8-PLT-36-LACA,LACA.8-PLT-36-LACA,38.143611,-77.855555,LACA,1.0,0.3,17.0,nan,nan,8.52,nan,nan,9.33,nan,nan,nan,nan,7.29,nan,1.5,nan,0.650,0.06,nan,nan,19.12,None,None,None,None,None,None,None,None,None,None,None,14.97052,0.0,4.003388,4.235404,5.208091,20801060503
1,2014-10-07 00:00:00,10:00:00,EX-1,LACA.EX-1,38.114880,-78.010120,LACA,1.0,0.3,nan,nan,nan,18.6,0.048,nan,6.77,nan,nan,nan,nan,6.35,nan,nan,nan,0.600,0.03,nan,nan,nan,None,None,None,None,None,None,None,None,None,None,None,14.34552,0.0,2.7768254,3.344779,5.82062,20801060402
2,2019-03-24 00:00:00,15:20:00,BRCR1,NWA.BRCR1,38.564500,-75.672300,NWA,1.0,0.5,nan,nan,nan,13.9,nan,119.2,11.5,nan,nan,nan,nan,nan,0.1,1.3,nan,6.210,0.05,nan,nan,11.1,None,None,None,None,Low,None,None,Ripple,Sunny,None,None,19.64972,0.0,1.519013,2.7265177,6.236149,20801090205
3,2017-04-09 00:00:00,17:40:00,COBR1,NWA.COBR1,38.642200,-75.606800,NWA,1.0,0.5,nan,nan,nan,16.21,nan,105.0,10.5,nan,nan,nan,nan,nan,0.1,1.1,nan,4.590,0.05,nan,nan,15.9,None,None,None,None,High,None,None,Ripple,Sunny,None,None,13.587219,0.0,15.3237,12.210892,13.09209,20801090404
4,2019-06-30 00:00:00,17:35:00,DEHE3,NWA.DEHE3,38.664700,-75.563900,NWA,1.0,0.5,nan,nan,nan,2.08,nan,92.2,7.3,nan,nan,nan,nan,nan,0.1,0.7,nan,4.228,0.035,nan,nan,27.2,None,None,None,None,High,None,None,Ripple,Sunny,None,None,10.71579,0.04910195,0.0050688675,1.5758352,2.195765,20801090404


### 1. Let's see when & where there are measurements of air temperature from the CMC sampling data

In [7]:
atdf=alldatadf.iloc[np.where(alldatadf.airtemp!="nan")][['Date','Time','Latitude','Longitude','airtemp']] 

In [8]:
atdf.sort_values(by='Date').head()

,Date,Time,Latitude,Longitude,airtemp
2045,2013-04-16 00:00:00,09:25:00,38.053611,-77.765555,15.1
1852,2013-04-16 00:00:00,09:56:00,38.053611,-77.841389,20.0
2308,2013-04-16 00:00:00,09:00:00,38.025278,-77.806944,13.0
1846,2013-04-16 00:00:00,09:00:00,38.143611,-77.855555,15.0
2032,2013-04-16 00:00:00,10:00:00,37.984722,-77.766944,17.5


In [9]:
atdf.sort_values(by='Date').tail()

,Date,Time,Latitude,Longitude,airtemp
873,2019-10-08 00:00:00,09:00:00,38.141667,-77.929722,14.5
2073,2019-10-08 00:00:00,10:20:00,38.143611,-77.855555,15.0
1938,2019-10-08 00:00:00,09:00:00,38.087083,-77.783861,22.5
895,2019-10-08 00:00:00,09:40:00,38.162780,-77.906940,15.5
1273,2019-10-08 00:00:00,09:05:00,38.025278,-77.806944,13.0


In [10]:
at_datetime=pd.to_datetime(atdf['Date'].astype(str) + ' ' + atdf['Time']).reset_index()

In [11]:
#data ranges
start_year=min(at_datetime).year
end_year=max(at_datetime).year
latrange=([min(alldatadf.Latitude), max(alldatadf.Latitude)])
lonrange=([min(alldatadf.Longitude), max(alldatadf.Longitude)])

print("year range:", start_year,"to",end_year)
print("lat range:", latrange)
print("lon range:", lonrange)

year range: 2013 to 2019
lat range: [37.02447, 42.37778]
lon range: [-79.89764, -75.40639]

### Plot sampling locations

In [13]:
# make a geo data frame including the points of all the station locations
gdf=geopandas.GeoDataFrame(atdf,
                           geometry=geopandas.points_from_xy(
                             atdf.Longitude.astype(float),atdf.Latitude.astype(float)))
tn_gdf=geopandas.GeoDataFrame(alldatadf,
                           geometry=geopandas.points_from_xy(
                             alldatadf.Longitude.astype(float),alldatadf.Latitude.astype(float)))

In [14]:
gdf.head()

,Date,Time,Latitude,Longitude,airtemp,geometry
0,2015-04-21 00:00:00,09:00:00,38.143611,-77.855555,17.0,POINT (-77.85555 38.14361)
12,2019-09-25 00:00:00,13:57:00,39.459333,-75.880056,26.5,POINT (-75.88006 39.45933)
27,2018-06-05 00:00:00,09:00:00,38.025278,-77.806944,25.2,POINT (-77.80694 38.02528)
45,2014-04-16 00:00:00,10:25:00,37.984722,-77.766944,5.5,POINT (-77.76694 37.98472)
47,2019-06-04 00:00:00,10:40:00,38.143611,-77.855555,22.5,POINT (-77.85555 38.14361)


In [15]:
gdf.count()

Out[64]: Date 226
Time 226
Latitude 226
Longitude 226
airtemp 226
geometry 226
dtype: int64

In [16]:
# import a map of US states
fid=urllib.request.urlretrieve("https://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_500k.json",
                               "/tmp/gz_2010_us_040_00_500k.json") 
us_states = geopandas.read_file(fid[0])

In [17]:
# make a map of sampling locations

plt.rcParams.update({'font.size': 16})

ax = us_states.boundary.plot(color='black',figsize=[7,14])

tn_gdf.plot(ax=ax, color='blue',marker='.')
gdf.plot(ax=ax, color='red')

minx, miny, maxx, maxy = gdf.total_bounds
ax.set_xlim(minx-1, maxx+1)
ax.set_ylim(miny-1, maxy+1)
ax.set_title('CMC sampling locations with TN (blue) & Air Temp (red) \n N=226')

plt.show()

### Plot sampling times

In [19]:
plt.rcParams.update({'font.size': 14})
plt.plot(at_datetime,atdf.airtemp.astype(float),'.')
plt.xlabel('Date')
plt.ylabel('Air Temperature')
plt.show()

### 2. Download data from NARR FTP

In [21]:
# get lats & lons for NARR dataset
# pick any year, any one of the files you'll be using later 
i=2013
fid=urllib.request.urlretrieve("ftp://ftp.cdc.noaa.gov/Datasets/NARR/monolevel/air.2m."+str(i)+".nc","/tmp/air.2m."+str(i)+".nc") 
nc_fid = Dataset(fid[0],'r')
narrlats = nc_fid.variables['lat'][:].data  
narrlons = nc_fid.variables['lon'][:].data

### Define a function to extract our region of interest from the NARR data

In [23]:
#Define a function to extract just the Chesapeake Bay region 

def extractCB(vardata,latdata,londata,latrange,lonrange):
  latlon_inds=np.where((latdata>latrange[0]) & (latdata<latrange[1])
                       & (londata>lonrange[0]) & (londata<lonrange[1]))
  var_subset=vardata[:,latlon_inds[0].min():latlon_inds[0].max(),latlon_inds[1].min():latlon_inds[1].max()]
  lat_subset=latdata[latlon_inds[0].min():latlon_inds[0].max(),latlon_inds[1].min():latlon_inds[1].max()]
  lon_subset=londata[latlon_inds[0].min():latlon_inds[0].max(),latlon_inds[1].min():latlon_inds[1].max()]
  return lat_subset, lon_subset, var_subset

### Define a function to loop through and return the subsetted area at the times of interest

In [25]:
# Define a function to loop through and return the subsetted area at the times of interest

def narr_subset_variable(file_prefix,varname,start_year,end_year,narr_lats,narr_lons,latrange,lonrange): 
  for i in range(start_year,end_year+1):
    fid=urllib.request.urlretrieve("ftp://ftp.cdc.noaa.gov/Datasets/NARR/monolevel/"+file_prefix+str(i)+".nc",
                                   "/tmp/"+file_prefix+str(i)+".nc") 
    nc_fid = Dataset(fid[0])
    if i==start_year:
      time = nc_fid.variables['time'][:].data
      subset_time = [dt.datetime(1800,1,1) + dt.timedelta(hours=t) for t in time]
      vartmp = nc_fid.variables[varname][:]  # shape is time, lat, lon as shown above
      latsubset, lonsubset, varsubset=extractCB(vartmp.data,narr_lats,narr_lons,latrange,lonrange)
      print(i,varname,'data imported')
    else:
      time = nc_fid.variables['time'][:].data
      dt_tmp = [dt.datetime(1800,1,1) + dt.timedelta(hours=t) for t in time]
      subset_time = np.append(subset_time,dt_tmp)
      vartmp = nc_fid.variables[varname][:]  # shape is time, lat, lon as shown above
      _, _, varsubsettmp=extractCB(vartmp.data,narr_lats,narr_lons,latrange,lonrange)
      varsubset=np.vstack((varsubset,varsubsettmp))
      print(i,varname,'data imported')
  return varsubset, subset_time, latsubset, lonsubset

In [26]:
# specify inputs and run the subset function for air temperature
file_prefix='air.2m.'
varname='air'
airsubset, time_narr_import, lat_subset, lon_subset=narr_subset_variable(
  file_prefix,varname,start_year,end_year,narrlats,narrlons,latrange,lonrange)

2013 air data imported
2014 air data imported
2015 air data imported
2016 air data imported
2017 air data imported
2018 air data imported
2019 air data imported

### Plot an array with shapefile and discrete points

In [28]:
ax = us_states.boundary.plot(color='black',figsize=[7,14])

minx, miny, maxx, maxy = gdf.total_bounds
ax.set_xlim(minx-1, maxx+1)
ax.set_ylim(miny-1, maxy+1)

ax.pcolor(lon_subset, lat_subset, airsubset[1,:,:])
tn_gdf.plot(ax=ax, color='blue',marker='.')
gdf.plot(ax=ax, color='red')

### 3. Compare the NARR data to the sampling location data

In [30]:
# find the time indices in the NARR timeseries corresponding to CMC and CBP measurements

# make a dataframe and make the narr timestamps the index
narrtdf=pd.DataFrame({'Datetime':pd.DatetimeIndex(time_narr_import)})
narrtdf=narrtdf.set_index('Datetime')

# make a new empty array for the time indices
narr_tind=np.array([],dtype=int) 

for i in range(len(at_datetime)):
  ttmp=at_datetime.iloc[i,1]
  #if CMC/CBP data are newer than the last timestamp for NARR data
  if (ttmp>narrtdf.index[-1]): 
    tmpind=-1
  else:
    tmpind=narrtdf.index.get_loc(ttmp,method='bfill')
  narr_tind=np.append(narr_tind,tmpind)
  
# save narr_tind as integer 
narr_tind=narr_tind.astype(int)

In [31]:
# first, combine the CMC coordinates and zip location data

#CMC coordinates 
allcoord = list(zip(atdf.Latitude,atdf.Longitude))

#NARR coordinates 
narrcoord = list(zip(np.ravel(lat_subset), np.ravel(lon_subset)))

In [32]:
narrcoord[:10]

Out[192]: [(37.61985, -81.82977),
 (37.52493, -81.48993),
 (37.42883, -81.15118),
 (37.33154, -80.81352),
 (37.23308, -80.47698),
 (37.13345, -80.14156),
 (37.03266, -79.80726),
 (36.93072, -79.4741),
 (36.82763, -79.14207),
 (36.72341, -78.81119)]

In [33]:
# then, find the closest NARR grid cell to the CMC coordinates 

# make a kdtree for the narr coordinates, then query it for the CMC coordinates
tree = spatial.KDTree(narrcoord)

# query and obtain corresponding climate data
narrxyid=tree.query(allcoord,k=1)

#get the matrix indices for the re-rolled matrix to correspond to the airtemp data
narrxy_ind1=(np.floor(narrxyid[1]/lat_subset.shape[1])).astype(int)
narrxy_ind2=(narrxyid[1]-narrxy_ind1*lat_subset.shape[1]).astype(int)

In [34]:
# extract air temp and convert kelvin to celsius
narr_cmc_airtemp=airsubset[narr_tind,narrxy_ind1,narrxy_ind2]-273.15

In [35]:
x=atdf.airtemp.astype(float)
y=narr_cmc_airtemp.astype(float)

plt.xlim(-5, 35)
plt.ylim(-5, 35)
plt.plot(x, y,'o',color = 'k')
ax.set_aspect('auto')

plt.title('CMC and NARR air temp comparison')
plt.xlabel('CMC air temp')
plt.ylabel('NARR air temp')

plt.show()